In [1]:
# basic spark set up 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
spark.version

'3.0.0'

In [3]:
df = spark.read.csv('/home/fung/Downloads/developer_survey_2020/survey_results_public.csv'
                    , header=True
                   , inferSchema=True)

In [4]:
# get to know the row and columns, strangely there is no df.shape as in pandas
print((df.count(), len(df.columns)))

(64461, 61)


## Data Exploration

In [5]:
import pyspark.sql.functions as f
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pyspark.sql

### 1. Not surprisingly most people start before 20. But quite a few started even before in University

In [6]:
d = {
    'ConvertedComp': IntegerType(),
    'Age1stCode': IntegerType(),
    'YearsCodePro': IntegerType(),
    'WorkWeekHrs': IntegerType()
}

for k, v in d.items():   
    df = df.withColumn(k, df[k].cast(v))

#### when a developer start really young(below 15), it is almost certain his or her comensation will be above average. surprisingly people pick up coding after typical university years will have better pay than those took CS in uni?

In [7]:
# group age 1st code into age range
from pyspark.sql.functions import udf

age_range = udf(lambda age: 'Nan' if age is None else
                    '<10' if age < 10 else 
                   '10-15' if (age >= 10 and age < 15) else
                   '15-20' if (age >= 15 and age < 20) else
                   '20-25' if (age >= 20 and age < 25) else
                   '25-30' if (age >= 25 and age < 30) else
                   '30-35' if (age >= 30 and age < 35) else
                   '35+' if (age > 35)
                    else 'Nan')

years_coded_range = udf(lambda year_coded: 'Nan' if year_coded is None else
                    '<2' if year_coded < 2 else 
                   '2-4' if (year_coded >= 2 and year_coded < 4) else
                   '4-6' if (year_coded >= 4 and year_coded < 6) else
                   '6-8' if (year_coded >= 6 and year_coded < 8) else
                   '8-10' if (year_coded >= 8 and year_coded < 10) else
                   '10+' if (year_coded > 10)
                    else 'Nan')

df2 = df.withColumn('age_range', age_range(df.Age1stCode))
df2 = df2.withColumn('year_coded_range', years_coded_range(df2.YearsCodePro))
# df2.groupBy('age_range').count().sort(desc('count')).show()
df2 = df2 \
    .groupBy('age_range', 'DevType')\
    .agg(
        {'age_range':'count', 
         'ConvertedComp': 'avg',
         'YearsCodePro': 'avg',
        })\
    .sort(desc('avg(ConvertedComp)'))\
    .show()


+---------+--------------------+----------------+------------------+-----------------+
|age_range|             DevType|count(age_range)|avg(ConvertedComp)|avg(YearsCodePro)|
+---------+--------------------+----------------+------------------+-----------------+
|    15-20|Academic research...|               1|         1920000.0|              3.0|
|    25-30|Developer, back-e...|               2|         1860000.0|              6.0|
|    10-15|Developer, back-e...|               1|         1860000.0|             24.0|
|      <10|Data scientist or...|               1|         1800000.0|              9.0|
|      <10|Data or business ...|               1|         1800000.0|             19.0|
|    10-15|Database administ...|               2|         1800000.0|             18.5|
|    10-15|Developer, back-e...|               2|         1680000.0|              9.0|
|    10-15|Developer, deskto...|               1|         1620000.0|             10.0|
|    15-20|Database administ...|           

### 2. Top 10 desire tech next year, pretty general stuff. 
    Oracle seems really obsoleted now
    Python is the top of the list, no doubt, hurray :)
    Rust is worth a look , also golang. TypeScript is very unncommon in HK but guess the trend is gravtating there
    Node.js got a lot of mentions, though only thing relvant to me is TensorFlow
    Webframework is obviously the realm of JS. In python it is still dominated by Django and Flask, personally more partial to flask.

In [8]:
desire_cols = [c for c in df.schema.names if 'Desire' in c]

for c in desire_cols:
    df_desire = df.filter(df[c] != 'NA')
    df_desire.groupby(c).count().sort(desc('count')).show(10, truncate=False)

+----------------------+-----+
|DatabaseDesireNextYear|count|
+----------------------+-----+
|PostgreSQL            |2310 |
|Microsoft SQL Server  |1947 |
|MySQL                 |1823 |
|MongoDB               |1624 |
|SQLite                |966  |
|Firebase              |901  |
|PostgreSQL;Redis      |787  |
|PostgreSQL;SQLite     |689  |
|MongoDB;MySQL         |667  |
|MongoDB;PostgreSQL    |576  |
+----------------------+-----+
only showing top 10 rows

+-------------------------------------+-----+
|LanguageDesireNextYear               |count|
+-------------------------------------+-----+
|Python                               |1152 |
|Rust                                 |528  |
|HTML/CSS;JavaScript;TypeScript       |499  |
|C#                                   |461  |
|Go                                   |412  |
|HTML/CSS;JavaScript                  |410  |
|C#;HTML/CSS;JavaScript;SQL;TypeScript|401  |
|Java                                 |340  |
|JavaScript                       

### 3. Comnpensation by Country: there must be some input error or super outliner in Lao. Anyway, countray to popular thought, Norway has such high average working hours


In [9]:
# orderd by avg comp desc
from pyspark.sql.functions import *
df3 = df\
    .groupBy('Country')\
    .agg(
        avg('ConvertedComp').alias('avg_comp'), 
        avg('WorkWeekHrs').alias('avg_working_hr')
    )\
    .orderBy('avg_comp', ascending=False)
df3 = df3.withColumn('hourly_wage', df3.avg_comp / df3.avg_working_hr/50)
df3 = df3.withColumn('hourly_wage', round(col('hourly_wage'), 2))\
        .withColumn('avg_comp', round(col('avg_comp'), 2))\
        .withColumn('avg_working_hr', round(col('avg_working_hr'), 2))
df3.show(truncate = False) 

#order by avg_working_hr asc
df3.na.drop().orderBy('hourly_wage', ascending=False).show()

+--------------------------------+---------+--------------+-----------+
|Country                         |avg_comp |avg_working_hr|hourly_wage|
+--------------------------------+---------+--------------+-----------+
|Lao People's Democratic Republic|231275.25|43.0          |107.57     |
|Nomadic                         |180995.23|41.7          |86.81      |
|United States                   |147844.08|41.79         |70.75      |
|Ireland                         |143952.98|39.56         |72.77      |
|Norway                          |127266.29|55.4          |45.94      |
|United Kingdom                  |119301.67|39.12         |60.99      |
|New Zealand                     |110159.19|40.5          |54.4       |
|Switzerland                     |109950.28|39.58         |55.55      |
|Australia                       |105617.38|39.81         |53.06      |
|Republic of Korea               |103590.62|35.14         |58.96      |
|Israel                          |103550.44|44.74         |46.29

### 4. so is there any pattern of developer type and education level? Master degree is the most common educate level for Academic researcher and Data scientist, while the rest are Bachelor. Guess that fit everyone expectation.

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

In [11]:
from pyspark.sql.functions import arrays_zip, col, explode, split
df4 = df.withColumn('DevType_splitted', explode(split('DevType', ';')))
df4 = df4.groupBy('DevType_splitted','edLevel').count().orderBy('DevType_splitted')
w = Window().partitionBy("DevType_splitted").orderBy(col("count").desc())
df_devtype_by_edlevel = (df4
  .withColumn("rn", row_number().over(w))
  .where(col("rn") == 1)
  .select("DevType_splitted", "edLevel", "count"))
df_devtype_by_edlevel.show()

+--------------------+--------------------+-----+
|    DevType_splitted|             edLevel|count|
+--------------------+--------------------+-----+
| Academic researcher|Master’s degree (...| 1171|
|Data or business ...|Bachelor’s degree...| 1773|
|Data scientist or...|Master’s degree (...| 1529|
|Database administ...|Bachelor’s degree...| 2474|
|            Designer|Bachelor’s degree...| 2406|
|   DevOps specialist|Bachelor’s degree...| 2702|
|Developer, QA or ...|Bachelor’s degree...| 1862|
| Developer, back-end|Bachelor’s degree...|13218|
|Developer, deskto...|Bachelor’s degree...| 5455|
|Developer, embedd...|Bachelor’s degree...| 1973|
|Developer, front-end|Bachelor’s degree...| 9106|
|Developer, full-s...|Bachelor’s degree...|13510|
|Developer, game o...|Bachelor’s degree...| 1103|
|   Developer, mobile|Bachelor’s degree...| 4627|
|            Educator|Bachelor’s degree...| 1085|
|      Engineer, data|Bachelor’s degree...| 1594|
|Engineer, site re...|Bachelor’s degree...|  898|


### 5. How common is DevOps in the industy? Assuminkg people with NA also considered as no DevOps, it isn't as common as I thought.

In [12]:
df_devops = df.groupBy('NEWDevOps')\
            .count().orderBy('count', ascending=False)\
            .show()

+---------+-----+
|NEWDevOps|count|
+---------+-----+
|       NA|21775|
|      Yes|18712|
|       No|18630|
| Not sure| 5344|
+---------+-----+

